In [1]:
import numpy as np
import random
import networkx as nx
from matplotlib import pyplot as plt
import pandas as pd
import copy
import os
import sys
import networkx as nx
import seaborn as sns
from scipy.io import mmread
sns.set(style='darkgrid')
sns.set_style(style='whitegrid')

In [2]:
# moduleー
current_dir = os.path.dirname(os.path.abspath("__file__"))
sys.path.append( str(current_dir) + '/../' )

In [3]:
from setting_param import MakeSample_prediction_num_of_edge_InputDir
from setting_param import L
InputDir = MakeSample_prediction_num_of_edge_InputDir

In [4]:
# READ EXIST_TABLE
EXIST_TABLE = np.load(InputDir + '/exist_table.npy')
EXIST_TABLE.shape

(8077, 16)

In [5]:
def ExistNodeList(ts):
    assert ts >= 0, "ts < 0 [referrence error]"
    return np.where(EXIST_TABLE[:, ts]==1)[0]

def GetAppearedNodes(ts):
    return set(ExistNodeList(ts)) - set(ExistNodeList(ts-1))

def GetObservedNodes(ts, L):
    U = set()
    for i in range(L):
        U |= set(ExistNodeList(ts-i))
    return U

def GetNodes(ts, L, node_type):
    if node_type=='all':
        node_set = set(ExistNodeList(ts))
    elif node_type=='stay':
        node_set = set(ExistNodeList(ts-1)) & set(ExistNodeList(ts))
    elif node_type=='lost':
        node_set = set(ExistNodeList(ts-1)) - set(ExistNodeList(ts))
    elif node_type=='return':
        node_set = GetAppearedNodes(ts) - (GetAppearedNodes(ts) - GetObservedNodes(ts-1, L))
    elif node_type=='new':
        node_set = GetAppearedNodes(ts) - GetObservedNodes(ts-1, L)
        node_set |= GetNodes(ts, L, 'return')
    return node_set

In [6]:
def Nx(ts):
    return  nx.from_numpy_matrix(mmread(InputDir + '/adjacency' + str(ts)).toarray())

def SubNxNew(ts, L):
    return nx.Graph(Nx(ts).edges(GetNodes(ts, L, 'new')))

def SubNxLost(ts, L):
    return nx.Graph(Nx(ts-1).edges(GetNodes(ts, L, 'lost')))

def GetEdges(ts, L, edge_type):
    G_1 = Nx(ts)
    if edge_type == "all":
        edge_set = G_1.edges
    elif edge_type == 'stay':
        G_0 = Nx(ts - 1)
        edge_set = G_0.edges & G_1.edges
    elif edge_type == "appeared":
        G_0 = Nx(ts - 1)
        edge_set = G_1.edges - G_0.edges - SubNxNew(ts, L).edges
    elif edge_type == "disappeared":
        G_0 = Nx(ts - 1)
        edge_set = G_0.edges - G_1.edges - SubNxLost(ts, L).edges
    return edge_set

In [7]:
def TsSplit(ts, L):
    ts_train = [(ts+l) for l in range(L)]
    ts_test = ts_train[-1]+1
    ts_all = ts_train.copy()
    ts_all.extend([ts_test])
    return ts_train, ts_test, ts_all

In [8]:
links = []
for ts in range(L, EXIST_TABLE.shape[1]):
    links.extend(list(GetEdges(ts, L, 'all')))
len(set(links))

22375

In [9]:
#  (0, 1return node)
n_links = []
for ts in range(L-1, EXIST_TABLE.shape[1]):
    n_links.extend(list(GetEdges(ts, L, 'all')))
len(set(n_links))

23554

In [10]:
#  (0, 1return node)
n_nodes = []
for ts in range(L-1, EXIST_TABLE.shape[1]):
    n_nodes.extend((GetNodes(ts, L, 'all')))
len(set(n_nodes))

7756

In [11]:
n_links = []
for ts in range(L-1, EXIST_TABLE.shape[1]):
    n_links.append(len(GetEdges(ts, L, 'all')))
n_links

[2518,
 2618,
 2620,
 2614,
 2640,
 2611,
 2720,
 2623,
 2885,
 2597,
 2935,
 2690,
 2777,
 2761]

In [12]:
n_nodes = []
for ts in range(L-1, EXIST_TABLE.shape[1]):
    n_nodes.append(len(GetNodes(ts, L, 'all')))
n_nodes

[1821,
 1901,
 1923,
 1890,
 1983,
 1954,
 2037,
 2000,
 2061,
 1940,
 2123,
 2082,
 2084,
 2012]